# 5 lesser-known pandas tricks

In [1]:
from platform import python_version

import pandas as pd
import xlsxwriter

## Setup

In [2]:
print("python version==%s" % python_version())
print("pandas==%s" % pd.__version__)
print("xlsxwriter==%s" % xlsxwriter.__version__)

python version==3.7.3
pandas==0.25.3
xlsxwriter==1.2.6


## Date Ranges

In [3]:
date_from = "2019-01-01"
date_to = "2019-01-12"
date_range = pd.date_range(date_from, date_to, freq="D")
date_range

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12'],
              dtype='datetime64[ns]', freq='D')

In [4]:
for i, (date_from, date_to) in enumerate(zip(date_range[:-1], date_range[1:]), 1):
    date_from = date_from.date().isoformat()
    date_to = date_to.date().isoformat()
    print("%d. date_from: %s, date_to: %s" % (i, date_from, date_to))

1. date_from: 2019-01-01, date_to: 2019-01-02
2. date_from: 2019-01-02, date_to: 2019-01-03
3. date_from: 2019-01-03, date_to: 2019-01-04
4. date_from: 2019-01-04, date_to: 2019-01-05
5. date_from: 2019-01-05, date_to: 2019-01-06
6. date_from: 2019-01-06, date_to: 2019-01-07
7. date_from: 2019-01-07, date_to: 2019-01-08
8. date_from: 2019-01-08, date_to: 2019-01-09
9. date_from: 2019-01-09, date_to: 2019-01-10
10. date_from: 2019-01-10, date_to: 2019-01-11
11. date_from: 2019-01-11, date_to: 2019-01-12


## Merge with indicator

In [5]:
left = pd.DataFrame({"key": ["key1", "key2", "key3", "key4"], "value_l": [1, 2, 3, 4]})
left

,key,value_l
0,key1,1
1,key2,2
2,key3,3
3,key4,4


In [6]:
right = pd.DataFrame({"key": ["key3", "key2", "key1", "key6"], "value_r": [3, 2, 1, 6]})
right

,key,value_r
0,key3,3
1,key2,2
2,key1,1
3,key6,6


In [7]:
df_merge = left.merge(right, on='key', how='left', indicator=True)
df_merge

,key,value_l,value_r,_merge
0,key1,1,1.0,both
1,key2,2,2.0,both
2,key3,3,3.0,both
3,key4,4,NaN,left_only


In [8]:
df_merge._merge.value_counts()

both          3
left_only     1
right_only    0
Name: _merge, dtype: int64

## Nearest merge

In [9]:
quotes = pd.DataFrame(
    [
        ["2016-05-25 13:30:00.023", "GOOG", 720.50, 720.93],
        ["2016-05-25 13:30:00.023", "MSFT", 51.95, 51.96],
        ["2016-05-25 13:30:00.030", "MSFT", 51.97, 51.98],
        ["2016-05-25 13:30:00.041", "MSFT", 51.99, 52.00],
        ["2016-05-25 13:30:00.048", "GOOG", 720.50, 720.93],
        ["2016-05-25 13:30:00.049", "AAPL", 97.99, 98.01],
        ["2016-05-25 13:30:00.072", "GOOG", 720.50, 720.88],
        ["2016-05-25 13:30:00.075", "MSFT", 52.01, 52.03],
    ],
    columns=["timestamp", "ticker", "bid", "ask"],
)
quotes['timestamp'] = pd.to_datetime(quotes['timestamp'])
quotes.shape

(8, 4)

In [10]:
quotes

,timestamp,ticker,bid,ask
0,2016-05-25 13:30:00.023,GOOG,720.50,720.93
1,2016-05-25 13:30:00.023,MSFT,51.95,51.96
2,2016-05-25 13:30:00.030,MSFT,51.97,51.98
3,2016-05-25 13:30:00.041,MSFT,51.99,52.00
4,2016-05-25 13:30:00.048,GOOG,720.50,720.93
5,2016-05-25 13:30:00.049,AAPL,97.99,98.01
6,2016-05-25 13:30:00.072,GOOG,720.50,720.88
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03


In [11]:
trades = pd.DataFrame(
    [
        ["2016-05-25 13:30:00.023", "MSFT", 51.95, 75],
        ["2016-05-25 13:30:00.038", "MSFT", 51.95, 155],
        ["2016-05-25 13:30:00.048", "GOOG", 720.77, 100],
        ["2016-05-25 13:30:00.048", "GOOG", 720.92, 100],
        ["2016-05-25 13:30:00.048", "AAPL", 98.00, 100],
    ],
    columns=["timestamp", "ticker", "price", "quantity"],
)
trades['timestamp'] = pd.to_datetime(trades['timestamp'])
trades.shape

(5, 4)

In [12]:
trades.head()

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:00.023,MSFT,51.95,75
1,2016-05-25 13:30:00.038,MSFT,51.95,155
2,2016-05-25 13:30:00.048,GOOG,720.77,100
3,2016-05-25 13:30:00.048,GOOG,720.92,100
4,2016-05-25 13:30:00.048,AAPL,98.00,100


In [13]:
pd.merge_asof(trades, quotes, on="timestamp", by='ticker', tolerance=pd.Timedelta('10ms'), direction='backward')

,timestamp,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


## Create an Excel report

In [14]:
df = pd.DataFrame(pd.np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=["a", "b", "c"])
df.shape

(3, 3)

In [15]:
report_name = 'example_report.xlsx'
sheet_name = 'Sheet1'

writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
df.to_excel(writer, sheet_name=sheet_name, index=False)
# writer.save() 

In [16]:
# define the workbook
workbook = writer.book
worksheet = writer.sheets[sheet_name]

# create a chart line object
chart = workbook.add_chart({'type': 'line'})

# configure the series of the chart from the spreadsheet
# using a list of values instead of category/value formulas:
#     [sheetname, first_row, first_col, last_row, last_col]
chart.add_series({
    'categories': [sheet_name, 1, 0, 3, 0],
    'values':     [sheet_name, 1, 1, 3, 1],
})

# configure the chart axes
chart.set_x_axis({'name': 'Index', 'position_axis': 'on_tick'})
chart.set_y_axis({'name': 'Value', 'major_gridlines': {'visible': False}})

# place the chart on the worksheet
worksheet.insert_chart('E2', chart)

# output the excel file
writer.save()

## Save the disk space

In [17]:
df = pd.DataFrame(pd.np.random.randn(50000,300))
df.shape

(50000, 300)

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.078273,-0.479193,-0.735543,-0.298239,1.329263,0.082488,-0.438242,1.326104,0.534224,1.741871,...,-0.124603,-0.543037,0.048416,0.494824,0.251238,-0.005348,-1.103462,-0.397582,0.599349,-1.328173
1,-0.541839,-0.909702,0.748465,0.765284,1.358393,1.816750,-1.065602,0.446391,0.014150,1.056500,...,-0.464392,0.537371,1.745196,0.159529,-0.966336,0.923511,-0.838287,0.169382,0.904446,1.832248
2,-0.860609,1.212723,-1.074181,0.601911,-1.022846,-0.396498,1.300595,1.779707,-0.559918,-0.285746,...,0.503387,1.016346,0.314884,-0.304264,0.804433,0.870342,-1.172353,1.305699,1.342068,-1.170054
3,-1.732922,-1.208682,0.974648,-0.550763,2.314935,0.788113,1.320727,-1.434199,-0.283080,1.058539,...,1.525849,0.091475,-0.450164,-0.607129,-0.609402,0.672017,-0.442552,-0.859368,-0.532058,-0.415734
4,-0.381423,-0.970862,-0.630154,-0.422975,-0.344676,-0.155697,-0.118481,0.626005,0.214405,1.121861,...,-0.356274,-0.337520,-0.046681,-0.923280,0.689289,0.915021,-0.149616,-0.491487,-0.175232,-0.463814


In [19]:
df.to_csv('random_data.csv', index=False)
df.shape

(50000, 300)

In [20]:
df.to_csv('random_data.gz', compression='gzip', index=False)
df.shape

(50000, 300)

In [21]:
df = pd.read_csv('random_data.gz')
df.shape

(50000, 300)